<a href="https://colab.research.google.com/github/varunkr24/Machine-Learning/blob/Python/Classify_Abnormal_BP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving TrainingDataAbnormalBloodPressure.csv to TrainingDataAbnormalBloodPressure.csv


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['TrainingDataAbnormalBloodPressure.csv']))

In [4]:
df.head()

,Patient_Number,Blood_Pressure_Abnormality,Level_of_Hemoglobin,Genetic_Pedigree_Coefficient,Age,BMI,Sex,Pregnancy,Smoking,Physical_activity,salt_content_in_the_diet,alcohol_consumption_per_day,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders
0,1,1,11.28,0.90,34,23,1,1.0,0,45961,48071,NaN,2,1,1
1,2,0,9.75,0.23,54,33,1,NaN,0,26106,25333,205.0,3,0,0
2,3,1,10.79,0.91,70,49,0,NaN,0,9995,29465,67.0,2,1,0
3,4,0,11.00,0.43,71,50,0,NaN,0,10635,7439,242.0,1,0,0
4,5,1,14.17,0.83,52,19,0,NaN,0,15619,49644,397.0,2,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Patient_Number                 2000 non-null   int64  
 1   Blood_Pressure_Abnormality     2000 non-null   int64  
 2   Level_of_Hemoglobin            2000 non-null   float64
 3   Genetic_Pedigree_Coefficient   1908 non-null   float64
 4   Age                            2000 non-null   int64  
 5   BMI                            2000 non-null   int64  
 6   Sex                            2000 non-null   int64  
 7   Pregnancy                      442 non-null    float64
 8   Smoking                        2000 non-null   int64  
 9   Physical_activity              2000 non-null   int64  
 10  salt_content_in_the_diet       2000 non-null   int64  
 11  alcohol_consumption_per_day    1758 non-null   float64
 12  Level_of_Stress                2000 non-null   i

In [6]:
df.isnull().mean()*100

Patient_Number                    0.0
Blood_Pressure_Abnormality        0.0
Level_of_Hemoglobin               0.0
Genetic_Pedigree_Coefficient      4.6
Age                               0.0
BMI                               0.0
Sex                               0.0
Pregnancy                        77.9
Smoking                           0.0
Physical_activity                 0.0
salt_content_in_the_diet          0.0
alcohol_consumption_per_day      12.1
Level_of_Stress                   0.0
Chronic_kidney_disease            0.0
Adrenal_and_thyroid_disorders     0.0
dtype: float64

In [7]:
df['Sex'].value_counts()

0    1008
1     992
Name: Sex, dtype: int64

In [8]:
df[df['Sex']==1]['Pregnancy'].isnull().sum()

550

In [9]:
df[df['Sex']==0]['Pregnancy'].isnull().sum()

1008

In [10]:
df['Pregnancy'].value_counts()

0.0    243
1.0    199
Name: Pregnancy, dtype: int64

In [11]:
df[(df['Sex']==1) & (df['Pregnancy'].isnull())]['Age'].agg(['min','max'])

min    45
max    75
Name: Age, dtype: int64

In [12]:
df[(df['Sex']==1) & (df['Pregnancy'] ==1)]['Age'].agg(['min','max'])

min    18
max    44
Name: Age, dtype: int64

Impute null values in Pregnancy with 0